In [3]:
async def generateVector():
    try:
        loader = PyPDFLoader("./mh_docs/masterigrandview.pdf")
        pages = loader.load()
        len(pages)

        # 2. Splitter
        text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_size=1000,
            chunk_overlap=150,
            length_function=len
        )

        docs = text_splitter.split_documents(pages)
        embeddings = OpenAIEmbeddings(
        model="text-embedding-3-large",
        )

        persist_directory = './docs/chroma/'

        vectordb = Chroma.from_documents(
            documents=docs,
            embedding=embeddings,
            persist_directory=persist_directory
        )

        return {"message": 'done'}
    except Exception as e:
        return {"error": str(e)}